## 1. Import Necessary Libraries

In [1]:
import pybullet as p
import pybullet_data
import time
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm

## 2. Configuration for Dataset Generation

This block defines the key parameters and constants for the data generation process. It specifies where the dataset will be saved, how much data to create, and the different text prompts to be associated with the robot's actions.

* **`DATASET_PATH`** and **`DATASET_NAME`**: These strings define the directory structure for saving the generated dataset.
* **`NUM_EPISODES`**: This sets the total number of simulation runs to be recorded. Each episode will represent one complete attempt at the task.
* **`VISUAL_CONFIRMATION`**: A boolean flag that controls how the simulation is run. If `True`, the simulation will run with a graphical user interface (GUI) so you can watch the robot. If `False`, it will run in headless mode (`p.DIRECT`), which is much faster and ideal for generating large amounts of data.
* **`PROMPTS`**: This is a list of different natural language commands that all describe the same task. Providing this **linguistic diversity** is crucial for training a robust model that can understand various ways a user might phrase a request.

In [2]:
DATASET_PATH = "datasets" 
DATASET_NAME = "my_robot_dataset"
NUM_EPISODES = 100
VISUAL_CONFIRMATION = True

PROMPTS = [
    "place the cube in the tray", "put the block into the container", "move the cube to the tray",
    "can you drop the cube inside the tray?", "pick up the cube and put it in the tray",
    "grasp the block and place it in the tray", "transfer the cube to the tray"
]


## 3. Dataset Generation Pipeline

This part defines a complete pipeline for generating a robotics dataset. It perform a pick-and-place task in a PyBullet simulation and saves the resulting trajectories (observations, actions, instructions) in the standardized `TFDS` (TensorFlow Datasets) format.

### 3.1. The Expert Policy (`get_expert_waypoints`)
This function acts as a perfect, hard-coded agent or "expert." It doesn't use AI; instead, it generates a deterministic sequence of waypoints (key positions and gripper states) to flawlessly solve the task. This scripted demonstration provides the ground-truth actions for our dataset. The sequence is logical:
1.  Open gripper.
2.  Move above the cube.
3.  Descend to grasp the cube.
4.  Close gripper.
5.  Lift the cube.
6.  Move above the tray.
7.  Descend to place the cube.
8.  Open gripper.
9.  Retract upwards.

### 3.2. The Episode Generator (`generate_episode_for_rlds`)
This is the main workhorse function that runs a single simulation episode and collects the data.
* **Smart Simulation Mode**: It cleverly runs the *first* episode in GUI mode for a quick visual check, then switches to the much faster headless (`DIRECT`) mode for the rest of the data generation.
* **Randomized Setup**: It creates a randomized scene for each episode (cube/tray positions and cube color) to ensure the dataset is diverse.
* **Data Recording Loop**: It follows the expert's waypoints. At each step, it records a dictionary containing:
    * **`observation`**
    * **`action`**: The action taken to reach the next state (change in position and gripper command).
    * **`language_instruction`**: A randomly chosen text prompt.
    * **Metadata**: Flags like `is_first` and `is_last` which are standard in RL datasets.

### 3.3. The TensorFlow Dataset Class (`MyRobotDataset`)
This class integrates our custom data generation logic with the `TFDS` framework.

In [3]:
def get_expert_waypoints(robot_id, cube_id, tray_id):
    """
    Generates a hard-coded sequence of waypoints for a perfect pick-and-place demonstration.
    This acts as the "expert" policy.
    """
    waypoints = []
    # Get the current positions of the key objects.
    cube_pos, _ = p.getBasePositionAndOrientation(cube_id)
    tray_pos, _ = p.getBasePositionAndOrientation(tray_id)
    robot_pos, _ = p.getBasePositionAndOrientation(robot_id)
    dx, dy = cube_pos[0] - robot_pos[0], cube_pos[1] - robot_pos[1]
    yaw_angle = np.pi / 2 if abs(dy) > abs(dx) else 0

    # Define a fixed orientation for the gripper during the task.
    target_orn = p.getQuaternionFromEuler([np.pi, 0, yaw_angle])

    # Define the precise position for grasping the cube.
    grasp_pos = np.array([cube_pos[0], cube_pos[1], cube_pos[2] - 0.015])

    # Define the sequence of movements and gripper actions.
    waypoints.extend([
        {'type': 'gripper', 'cmd': 0.0},  # 1. Start with gripper open.
        {'type': 'move', 'pos': np.array([cube_pos[0], cube_pos[1], cube_pos[2] + 0.2]), 'orn': target_orn}, # 2. Move above the cube.
        {'type': 'move', 'pos': grasp_pos, 'orn': target_orn}, # 3. Lower to grasp position.
        {'type': 'gripper', 'cmd': 1.0},  # 4. Close gripper.
        {'type': 'move', 'pos': np.array([cube_pos[0], cube_pos[1], cube_pos[2] + 0.2]), 'orn': target_orn}, # 5. Lift the cube up.
        {'type': 'move', 'pos': np.array([tray_pos[0], tray_pos[1], tray_pos[2] + 0.2]), 'orn': target_orn}, # 6. Move above the tray.
        {'type': 'move', 'pos': np.array([tray_pos[0], tray_pos[1], tray_pos[2] + 0.04]), 'orn': target_orn}, # 7. Lower to place position.
        {'type': 'gripper', 'cmd': 0.0},  # 8. Open gripper to release.
        {'type': 'move', 'pos': np.array([tray_pos[0], tray_pos[1], tray_pos[2] + 0.2]), 'orn': target_orn}  # 9. Retract upwards.
    ])
    return waypoints

def generate_episode_for_rlds(episode_idx):
    """
    Runs a full simulation episode, executes the expert policy, and records all data.
    """
    # Use GUI for the first episode for a visual check, then headless for speed.
    connection_mode = p.GUI if VISUAL_CONFIRMATION and episode_idx == 0 else p.DIRECT
    
    p.connect(connection_mode)
    p.setAdditionalSearchPath(pybullet_data.getDataPath()); p.setGravity(0, 0, -9.81)

    # If in GUI mode, configure the visualizer.
    if connection_mode == p.GUI:
        p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
        p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, 1)
        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING, 1)
        p.configureDebugVisualizer(p.COV_ENABLE_RGB_BUFFER_PREVIEW, 0)
        p.configureDebugVisualizer(p.COV_ENABLE_DEPTH_BUFFER_PREVIEW, 0)
        p.configureDebugVisualizer(p.COV_ENABLE_SEGMENTATION_MARK_PREVIEW, 0)
        p.resetDebugVisualizerCamera(cameraDistance=0.85,
                                cameraYaw=50,
                                cameraPitch=-35,
                                cameraTargetPosition=[0, 0, 0.7])
        time.sleep(1)

    p.loadURDF("plane.urdf")
    p.loadURDF("table/table.urdf", basePosition=[0, 0, 0])
    robot_id = p.loadURDF("franka_panda/panda.urdf", basePosition=[-0.45, 0, 0.625], useFixedBase=True)
    # Reset robot to a default pose.
    for i, angle in enumerate([0, -0.4, 0, -2.4, 0, 2.0, 0.8]):
        p.resetJointState(robot_id, i, angle)
    
    left_zone, right_zone = {'x': (0.1, 0.3), 'y': (0.1, 0.3)}, {'x': (0.1, 0.3), 'y': (-0.3, -0.1)}
    # Randomly assign cube and tray zones to ensure diversity.
    cube_zone, tray_zone = (left_zone, right_zone) if np.random.rand() > 0.5 else (right_zone, left_zone)
    # Randomize initial cube position within the defined zone.
    initial_cube_pos = [np.random.uniform(*cube_zone['x']), np.random.uniform(*cube_zone['y']), 0.7]
    cube_id = p.loadURDF("cube_small.urdf", basePosition=initial_cube_pos)
    # Randomize cube color for diversity.
    p.changeVisualShape(cube_id, -1, rgbaColor=[np.random.uniform(0.2, 0.8), np.random.uniform(0.2, 0.8), np.random.uniform(0.2, 0.8), 1.0])

    # Let the cube settle before getting its stable position.
    for _ in range(50):
        p.stepSimulation()

    stable_cube_pos, _ = p.getBasePositionAndOrientation(cube_id)
    # Randomize tray position within its zone.
    tray_pos = [np.random.uniform(*tray_zone['x']), np.random.uniform(*tray_zone['y']), 0.63]
    tray_id = p.loadURDF("tray/tray.urdf", basePosition=tray_pos, globalScaling=0.6)
    
    lang_instruction = np.random.choice(PROMPTS) # Pick a random instruction.
    waypoints = get_expert_waypoints(robot_id, cube_id, tray_id) # Get the expert plan
    episode_steps = []
    
     # Define camera properties for capturing images.
    view_matrix = p.computeViewMatrixFromYawPitchRoll(
        cameraTargetPosition=[0, 0, 0.7],
        distance=0.85,
        yaw=50,
        pitch=-35,
        roll=0,
        upAxisIndex=2
    )

    projection_matrix = p.computeProjectionMatrixFOV(fov=60, aspect=1.0, nearVal=0.01, farVal=3.0)

    for i, waypoint in enumerate(waypoints):
        # Record Observation
        link_state = p.getLinkState(robot_id, 11, computeForwardKinematics=True)
        current_pos, current_orn_quat = np.array(link_state[0]), np.array(link_state[1])
        current_gripper_state = 1.0 if p.getJointState(robot_id, 9)[0] < 0.02 else 0.0
        
        img_data = p.getCameraImage(224, 224, view_matrix, projection_matrix)
        image = np.reshape(img_data[2], (224, 224, 4))[:, :, :3]
        #save image
        # tf.keras.preprocessing.image.save_img("00.png", image)
        if waypoint['type'] == 'gripper':
            pos_delta = np.zeros(3)
            gripper_cmd = waypoint['cmd']
            gripper_target = 0.00 if waypoint['cmd'] > 0.5 else 0.04
            # Simulate for a few steps to complete the action.
            for _ in range(30):
                p.setJointMotorControl2(robot_id, 9, p.POSITION_CONTROL, gripper_target)
                p.setJointMotorControl2(robot_id, 10, p.POSITION_CONTROL, gripper_target)
                p.stepSimulation()
        else: # 'move' action
            # Action is the change in position.
            pos_delta = waypoint['pos'] - current_pos
            gripper_cmd = current_gripper_state
            joint_poses = p.calculateInverseKinematics(robot_id, 11, waypoint['pos'], waypoint['orn'])
            # Simulate for more steps to reach the target.
            for _ in range(60): 
                for j in range(7):
                    p.setJointMotorControl2(robot_id, j, p.POSITION_CONTROL, joint_poses[j], force=120)
                p.stepSimulation()
        # Store the complete step data
        episode_steps.append({
            'observation': {'image': image, 'proprio': np.concatenate([current_pos, p.getEulerFromQuaternion(current_orn_quat), [current_gripper_state]]).astype(np.float32)},
            'action': np.concatenate([pos_delta, [0,0,0], [gripper_cmd]]).astype(np.float32),
            'language_instruction': lang_instruction,
            'is_first': i == 0, 'is_last': i == len(waypoints) - 1, 'is_terminal': i == len(waypoints) - 1,
        })

    p.disconnect()
    
    # Return data for the whole episode.
    return f'episode_{episode_idx}', {
        'steps': episode_steps,
        'initial_state': {
            'cube_pos': np.array(stable_cube_pos).astype(np.float32),
            'tray_pos': np.array(tray_pos).astype(np.float32)
        }
    }

class MyRobotDataset(tfds.core.GeneratorBasedBuilder):
    """A TFDS builder for the generated robot dataset."""
    VERSION = tfds.core.Version('1.0.0')
    RELEASE_NOTES = {'1.0.0': 'Initial release for pick and place.'}

    def _info(self):
        return tfds.core.DatasetInfo(
            builder=self,
            features=tfds.features.FeaturesDict({
                'steps': tfds.features.Dataset({
                    'observation': tfds.features.FeaturesDict({'image': tfds.features.Image(shape=(224, 224, 3), dtype=tf.uint8), 'proprio': tfds.features.Tensor(shape=(7,), dtype=tf.float32)}),
                    'action': tfds.features.Tensor(shape=(7,), dtype=tf.float32), 'language_instruction': tfds.features.Text(),
                    'is_first': tfds.features.Scalar(dtype=tf.bool), 'is_last': tfds.features.Scalar(dtype=tf.bool), 'is_terminal': tfds.features.Scalar(dtype=tf.bool),
                }),
                'initial_state': tfds.features.FeaturesDict({
                    'cube_pos': tfds.features.Tensor(shape=(3,), dtype=tf.float32),
                    'tray_pos': tfds.features.Tensor(shape=(3,), dtype=tf.float32),
                })
            }),
        )
    def _split_generators(self, dl_manager: tfds.download.DownloadManager):
        return {'train': self._generate_examples()}

    def _generate_examples(self):
        """"Yields episodes for TFDS to write to disk."""
        print(f"Generating {NUM_EPISODES} episodes for RLDS dataset...")
        for i in tqdm(range(NUM_EPISODES)):
            yield generate_episode_for_rlds(i)


In [4]:
if not os.path.exists(DATASET_PATH):
    os.makedirs(DATASET_PATH)

builder = MyRobotDataset(data_dir=os.path.join(DATASET_PATH, DATASET_NAME))
builder.download_and_prepare()
print(f"\nRLDS dataset generated successfully at: {builder.data_dir}")

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Generating 100 episodes for RLDS dataset...







































































































100%|██████████| 100/100 [01:12<00:00,  1.39it/s]


Shuffling datasets\my_robot_dataset\my_robot_dataset\incomplete.EFJJ7Y_1.0.0\my_robot_dataset-train.tfrecord*.…

Dataset my_robot_dataset downloaded and prepared to datasets\my_robot_dataset\my_robot_dataset\1.0.0. Subsequent calls will reuse this data.

RLDS dataset generated successfully at: datasets\my_robot_dataset\my_robot_dataset\1.0.0


## 4. Verifying the Dataset: Replaying an Episode

This final part serves as a **validation tool**. Its purpose is to load a specific episode from the dataset we just created, and then "replay" the recorded actions in a new PyBullet simulation. This allows us to visually confirm that the data (initial states, observations, and actions) was saved correctly and that the recorded actions produce the expected behavior.

### The Replay Process (`replay_rlds_episode`)
The function works in four main stages:

1.  **Load the Dataset**
2.  **Select an Episode**
3.  **Reconstruct the Scene**
4.  **Playback Actions**

In [10]:
# The full path name for the generated TFDS dataset, including version.
DATASET_NAME = "my_robot_dataset/my_robot_dataset/1.0.0"

def replay_rlds_episode(episode_id_to_replay):
    """
    Loads and replays a specific episode from the saved TFDS dataset for visual verification.
    """
    
    full_dataset_path = os.path.join(DATASET_PATH, DATASET_NAME)
    print(f"Loading RLDS dataset from: {full_dataset_path}")

    # Load the dataset builder from the files on disk.
    try:
        builder = tfds.builder_from_directory(full_dataset_path)
    except Exception as e:
        print(f"Error loading dataset: {e}"); return
    
    # Get the 'train' split of the dataset.
    ds = builder.as_dataset(split='train')
    

    # Efficiently find the specific episode to replay.
    try:
        episode_to_replay = next(iter(ds.skip(episode_id_to_replay).take(1)))
        print(f"Successfully found episode {episode_id_to_replay}.")
    except (tf.errors.OutOfRangeError, StopIteration):
        print(f"Error: Episode with ID {episode_id_to_replay} not found."); return

    # Read the initial object positions from the loaded episode data.
    initial_state = episode_to_replay['initial_state']
    cube_pos = initial_state['cube_pos'].numpy()
    tray_pos = initial_state['tray_pos'].numpy()
    

    p.connect(p.GUI)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.81)
    p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
    p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, 1)
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING, 1)

    p.loadURDF("plane.urdf")
    p.loadURDF("table/table.urdf", basePosition=[0, 0, 0])
    robot_id = p.loadURDF("franka_panda/panda.urdf", basePosition=[-0.45, 0, 0.625], useFixedBase=True)
    for i, angle in enumerate([0, -0.4, 0, -2.4, 0, 2.0, 0.8]):
        p.resetJointState(robot_id, i, angle)
    
    p.resetDebugVisualizerCamera(cameraDistance=1.2, cameraYaw=70, cameraPitch=-35, cameraTargetPosition=[0.1, 0, 0.65])
    
    # Place the cube and tray at their recorded initial positions.
    cube_id = p.loadURDF("cube_small.urdf", basePosition=cube_pos)
    p.changeVisualShape(cube_id, -1, rgbaColor=[0.8, 0.2, 0.2, 1.0]) 
    tray_id = p.loadURDF("tray/tray.urdf", basePosition=tray_pos, globalScaling=0.6)

    print(f"Replaying Episode {episode_id_to_replay}...")
    time.sleep(2)

    for step in episode_to_replay['steps']:
        # Get the recorded action for this timestep.
        action_delta = step['action'].numpy()
        
        # Get the robot's current state to calculate the target state.
        link_state = p.getLinkState(robot_id, 11, computeForwardKinematics=True)
        current_pos, current_orn_quat = np.array(link_state[0]), np.array(link_state[1])
        
        pos_delta, orn_delta_rpy, gripper_cmd = action_delta[:3], action_delta[3:6], action_delta[6]
        
        # Calculate the target position and orientation.
        target_pos = current_pos + pos_delta
        
        orn_delta_quat = p.getQuaternionFromEuler(orn_delta_rpy)
        _, target_orn_quat = p.multiplyTransforms(current_pos, current_orn_quat, [0,0,0], orn_delta_quat)
        
        gripper_target = 0.00 if gripper_cmd > 0.5 else 0.04
        
         # Execute the action over a short duration for smooth visualization.
        for _ in range(30):
            joint_poses = p.calculateInverseKinematics(robot_id, 11, target_pos, target_orn_quat)
            for i in range(7):
                p.setJointMotorControl2(robot_id, i, p.POSITION_CONTROL, joint_poses[i], force=120)
            p.setJointMotorControl2(robot_id, 9, p.POSITION_CONTROL, gripper_target, force=50)
            p.setJointMotorControl2(robot_id, 10, p.POSITION_CONTROL, gripper_target, force=50)
            p.stepSimulation()
            time.sleep(1/240.)
            
    print("Replay finished.")
    time.sleep(3)
    p.disconnect()


In [12]:
replay_rlds_episode(10) # Change the episode ID as needed.

Loading RLDS dataset from: datasets\my_robot_dataset/my_robot_dataset/1.0.0
Successfully found episode 10.
Replaying Episode 10...
Replay finished.
